In [48]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.chat_models import init_chat_model
from langgraph.prebuilt import create_react_agent
from langchain.schema.messages import SystemMessage

# 1. Connect to DB
db = SQLDatabase.from_uri("sqlite:///data/northwind.db")  # Cleaned path

# 2. Initialize LLM
llm = init_chat_model("gpt-4o-mini", model_provider="openai")

# 3. Create toolkit and tools
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
tools = toolkit.get_tools()

# 4. Format prompt as SystemMessage
prompt_template = "You are an expert SQL agent. Use dialect: {dialect}. Return top {top_k} results."
system_message = SystemMessage(content=prompt_template.format(dialect="SQLite", top_k=5))

# 5. Create the agent
sql_agent = create_react_agent(model=llm, tools=tools, prompt=system_message)

# 6. Define query
query = "Identify the ten most expensive products (ProductName, UnitPrice)"

# 7. Stream the results
for event in sql_agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values"
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

Identify the ten most expensive products (ProductName, UnitPrice)
================================== Ai Message ==================================
Tool Calls:
  sql_db_list_tables (call_OYzzlYWIgZejboitMMmM8tko)
 Call ID: call_OYzzlYWIgZejboitMMmM8tko
  Args:
================================= Tool Message =================================
Name: sql_db_list_tables

Categories, CustomerCustomerDemo, CustomerDemographics, Customers, EmployeeTerritories, Employees, Order Details, Orders, Products, Regions, Shippers, Suppliers, Territories
================================== Ai Message ==================================
Tool Calls:
  sql_db_schema (call_uzsJ88zaSr1ByOi1tCdlx2qD)
 Call ID: call_uzsJ88zaSr1ByOi1tCdlx2qD
  Args:
    table_names: Products
================================= Tool Message =================================
Name: sql_db_schema


CREATE TABLE "Products" (
	"ProductID" INTEGER NOT NULL, 
	